In [3]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

In [11]:
# a different model
model2 = BayesianModel([('schooltype', 'deanlist'), 
                        ('schooltype', 'top20'), 
                        ('schooltype', 'gpa3'), 
                        ('schooltype', 'acttest'), 
                        ('deanlist', 'eligibility'), 
                        ('top20', 'eligibility'), 
                        ('gpa3', 'eligibility'), 
                        ('acttest', 'eligibility'), 
                        ('age', 'eligibility'), 
                        ('schooltype', 'eligibility')])  # fruit -> tasty <- size


# Defining individual CPDs.
# SchoolType   5 states   
# ACTtest      2 states    0-Yes, 1-No, 2-Unknown
# GPA          2 states    0-Yes, 1-No, 2-Unknown
# Top20        2 states    0-Yes, 1-No, 2-Unknown
# Deanlist     2 states    0-Yes, 1-No, 2-Unknown

cpd_schooltype = TabularCPD(variable='schooltype', variable_card=5, values=[[0.3, 0.3, 0.2, 0.15, 0.05]])
cpd_deanlist = TabularCPD(variable='deanlist', variable_card=3, values=[[0.45, 0.45, 0.1]])
cpd_top20 = TabularCPD(variable='top20', variable_card=3, values=[[0.45, 0.45, 0.1]])
cpd_gpa3 = TabularCPD(variable='gpa3', variable_card=3, values=[[0.45, 0.45, 0.1]])
cpd_acttest = TabularCPD(variable='acttest', variable_card=3, values=[[0.45, 0.45, 0.1]])
cpd_age = TabularCPD(variable='age', variable_card=3, values=[[0.45, 0.45, 0.1]])


cpd_deanlist = TabularCPD(variable='deanlist', variable_card=3, 
                   values=[[0.1,  0.15, 0.2,  0,  0],
                           [0.75, 0.75, 0.7,  0.8,  1],
                           [0.15,  0.1,  0.1,  0.2,  0]],
                  evidence=['schooltype'],
                  evidence_card=[5])


cpd_top20 = TabularCPD(variable='top20', variable_card=3, 
                   values=[[0.2, 0.2, 0.2,  0,  0],
                           [0.7, 0.7, 0.7,  1,  1],
                           [0.1, 0.1, 0.1,  0,  0]],
                   evidence=['schooltype'],
                   evidence_card=[5])


cpd_gpa3 = TabularCPD(variable='gpa3', variable_card=3, 
                   values=[[0.3, 0.2, 0.1,  0,  0],
                           [0.6, 0.7, 0.8,  1,  1],
                           [0.1, 0.1, 0.1,  0,  0]],
                   evidence=['schooltype'],
                   evidence_card=[5])


cpd_acttest = TabularCPD(variable='acttest', variable_card=3, 
                   values=[[0,   0,   0,    0.4,  0],
                           [0.8, 0.8, 0.8,  0.6,  1],
                           [0.2, 0.2, 0.2,  0,    0]],
                   evidence=['schooltype'],
                   evidence_card=[5])


cpd_eligibility = TabularCPD(variable='eligibility', variable_card=3, 
                   values=[var_true, var_false, var_unknown],
                   evidence=['schooltype','deanlist','top20','gpa3','acttest','age'],
                   evidence_card=[5,3,3,3,3,3])


model2.add_cpds(cpd_schooltype, cpd_deanlist, cpd_top20, cpd_gpa3, cpd_acttest, cpd_age, cpd_eligibility)


# rule set 1
def check_goodstudent(st, dl, t20, gpa3, acttest):
    if ((st==0) | (st==1) | (st==2)):
        if ((dl=='yes') | (t20=='yes') | (gpa3=='yes')):
            return 'yes'
        elif ((dl=='no') & (t20=='no') & (gpa3=='no')):  #all of them are no
            return 'no'
        else:
            return 'unknown'
    elif st==3:   # home school
        if (acttest=='yes'):
            return 'yes'
        elif (acttest=='no'):
            return 'no'
        else:  # don't know acctest result
            return 'unknown'
    else:
        return 'no'
    
    
    
# rule set 2
def check_eligibility(gs, age):
    if ((gs=='yes') & (age=='yes')):
        return 'yes'
    elif ((gs=='unknown') | (age=='unknown')):
        return 'unknown'
    else:
        return 'no'
    
    
# construct a 2-d array
var_true =[]
var_false =[]
var_unknown =[]

for st in range(0,5):  # 0 - Fulltime high school   1 - College  2- Trade School . 3- Homeschool . 4-Others
    for dl in ['yes', 'no', 'unknown']:  # 0 - Yes, on deanlist   1 - not on deanlist
        for t20 in ['yes', 'no', 'unknown']:   # 0 - Yes, is top 20%,  1 - not in top 20%
            for gpa3 in ['yes', 'no', 'unknown']:  # 0 - yes,  1 . -no
                for acttest in ['yes', 'no', 'unknown']:  # 0 - yes,  1 - no
                    for age in ['yes', 'no', 'unknown']: # 0 below 25,  1 - above 25
                        #print (check_goodstudent(st, dl, t20, gpa3, acttest))
                        if check_eligibility(check_goodstudent(st, dl, t20, gpa3, acttest),age)=='yes':
                            var_true.extend([1])
                            var_false.extend([0])
                            var_unknown.extend([0])
                        elif check_eligibility(check_goodstudent(st, dl, t20, gpa3, acttest),age)=='no':
                            var_true.extend([0])
                            var_false.extend([1])
                            var_unknown.extend([0])
                        else:
                            var_true.extend([0])
                            var_false.extend([0])
                            var_unknown.extend([1])                            
                
                



In [ ]:
# rank entropy
# evidence is a dictionary with known states

import math
from operator import itemgetter
from collections import OrderedDict
from pgmpy.inference import VariableElimination
infer = VariableElimination(model2)
print(infer.query(['eligibility']) ['eligibility'])


known_evidence={'deanlist':0}

def conditional_entropy(variable, my_evidence):
    cond_entropy = 0
    raw_p = infer.query([variable]) [variable].values
    card = len(raw_p)
    
    for v in range(0,card):
        # update the variable value to v
        my_evidence[variable] = v
        #  H(Z|x_j)*p(x_j)
        cond_entropy = cond_entropy + raw_p[v]* evidence_entropy(my_evidence)

    return cond_entropy

# def calculate_entropy_2(variable, my_evidence):
#     entropy = 0
#     my_evidence[variable] = 0
    
#     dis_fact = infer.query(['eligibility'], evidence=my_evidence) ['eligibility'].values
    
#     for v in dis_fact:
#         entropy =entropy - v*math.log(v+1E-10)
        
        
#     return entropy

def evidence_entropy(my_evidence):
    dis_fact = infer.query(['eligibility'], evidence=my_evidence) ['eligibility'].values
    entropy = 0
    for v in dis_fact:
        entropy =entropy - v*math.log(v+1E-10)
    return entropy

def calculate_gain(variable, my_evidence):
    H_z= evidence_entropy(my_evidence)
    H_z_x = conditional_entropy(variable, my_evidence)
#     print('old entropy' + str(H_z))
#     print('new entropy' + str(H_z_x))
    gain = H_z - H_z_x
    return gain

def rank_variable(my_evidence, variable_list):
    ranked_q = {}
    for var in variable_list:
        if var not in my_evidence:
            #print(var)
            #print(calculate_gain(var,my_evidence))
            ranked_q[var]=calculate_gain(var,my_evidence)
    return OrderedDict(sorted(ranked_q.items(), key=itemgetter(1),reverse=True))

#print(known_evidence)
#print(calculate_gain('age', known_evidence))

variable_list = ['schooltype','deanlist','top20','gpa3','acttest','age']
print(rank_variable(known_evidence,variable_list))


+---------------+--------------------+
| eligibility   |   phi(eligibility) |
|---------------+--------------------|
| eligibility_0 |             0.1937 |
| eligibility_1 |             0.5745 |
| eligibility_2 |             0.2318 |
+---------------+--------------------+
